In [1]:
import pandas as pd
import numpy as np
import json
import io
from io import StringIO
import os

from shapely.geometry import Point, LineString
from shapely.geometry.polygon import Polygon
from shapely import wkt

from math import sin, cos, sqrt, atan2, radians


C:\Users\olive\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\olive\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\olive\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Process
1. Collect geo-data
2. Calculate county centroid data
3. Clean up data frame with country centroid detail<br>

**Load clinic Data**<br>

4. Make all counties lower case
5. For each time period (2010 and 2015), generate 2 lists:
    * Counties with no clinic
    * Counties with clicic
6. For each time period, generate a list that finds closest county with a clinic. This is only for counties with no clinic in that time period
7. The resultant data frame has fields: Closest counties / Counties. Groupby Closest counties to see catchment area

**Estimate New Female Popultion with Catchment included**<br>

8. Create 2 columns per time peried
    * catchment area female pupulation that includes only catchment counties
    * 'new' total that includes catchment plus a counties own population


#### Texas Country Geo-Perimeter Data from:  
https://gis-txdot.opendata.arcgis.com/datasets/TXDOT::texas-county-boundaries-detailed/about


In [2]:
# Opening JSON file
f = open('.geojson')
# returns JSON object as
# a dictionary
data = json.load(f)

In [3]:
county_detail = []
for i in range(len(data['features'])):
    ft_obj = data['features'][i]['properties']
    county = ft_obj['CNTY_NM']
    polygon_temp = data['features'][i]['geometry']['coordinates'][0][0]
    line = LineString(polygon_temp)
    centroid = line.centroid.wkt
    county_detail.append({'county':county,'centroid':centroid})

In [4]:
county_detail = pd.DataFrame(county_detail)

In [5]:
county_detail.head()

,county,centroid
0,Atascosa,POINT (-98.52550129237503 28.89332098109323)
1,Matagorda,POINT (-95.99530169303894 28.84771261549633)
2,Jackson,POINT (-96.56923395706752 28.8874181215156)
3,De Witt,POINT (-97.36382516011945 29.07072652031978)
4,Wilson,POINT (-98.07891405628709 29.18641646756914)


### Clean up Data frame

In [6]:
## Remove 'POINT (' , trailing close parentasis, replace space for ,
county_detail['centroid'] = county_detail['centroid'].str.replace('POINT \(','')
county_detail['centroid'] = county_detail['centroid'].str.replace('\)','')
county_detail['centroid'] = county_detail['centroid'].str.replace(' ',',')
# create new lat / long fields
county_detail['longitude'] = county_detail['centroid'].astype(str).str.split(',').str[0]
county_detail['latitude'] = county_detail['centroid'].astype(str).str.split(',').str[1]
# make series float
county_detail['longitude'] = county_detail['longitude'].astype(float)
county_detail['latitude'] = county_detail['latitude'].astype(float)
# lower case counties
county_detail['county'] = [x.lower() for x in county_detail['county']]
# there was discprepency between 2 lists on dewitt county
county_detail['county'] = county_detail['county'].str.replace('de witt','dewitt')


C:\Users\olive\AppData\Local\Temp/ipykernel_13652/231644267.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  county_detail['centroid'] = county_detail['centroid'].str.replace('POINT \(','')
C:\Users\olive\AppData\Local\Temp/ipykernel_13652/231644267.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  county_detail['centroid'] = county_detail['centroid'].str.replace('\)','')


In [7]:
county_detail.head()

,county,centroid,longitude,latitude
0,atascosa,"-98.52550129237503,28.89332098109323",-98.525501,28.893321
1,matagorda,"-95.99530169303894,28.84771261549633",-95.995302,28.847713
2,jackson,"-96.56923395706752,28.8874181215156",-96.569234,28.887418
3,dewitt,"-97.36382516011945,29.07072652031978",-97.363825,29.070727
4,wilson,"-98.07891405628709,29.18641646756914",-98.078914,29.186416


### Load clinic data

In [8]:
clinic = pd.read_excel('contra&ab_ data.xlsx',sheet_name='Data_ for_STATA')

In [9]:
# clean trailing and leading spaces in 'counties
clinic['county'] = [x.strip() for x in clinic['county']]
clinic['county'] = [x.lower() for x in clinic['county']]

In [10]:
#columns = 'county','totclinic2010','totclin2015'
clnc_df = clinic[['county','totclinic2010','totclin2015']]

In [11]:
## Create 2 lists - one with counties with no clinics and the other with clinics
no_clc_list = list(clnc_df[clnc_df['totclinic2010'] == 0]['county'])
clc_list = list(clnc_df[clnc_df['totclinic2010'] >0 ]['county'])


In [12]:
len(no_clc_list)+len(clc_list)

254

In [13]:
# approximate radius of earth in km
def distance_earth(lat1,lon1,lat2,lon2):
    R = 3963.19

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [14]:
data_1_2010 = []
for cnty in no_clc_list:
    #print(cnty)
    #temp_cnty = cnty
    lat1 = county_detail[county_detail['county'] == cnty]['latitude'].values[0]
    lon1 = county_detail[county_detail['county'] == cnty]['longitude'].values[0]
    closest_distance = 1000
    for cnty2 in clc_list:
        lat2 = county_detail[county_detail['county'] == cnty2]['latitude'].values[0]
        lon2 = county_detail[county_detail['county'] == cnty2]['longitude'].values[0]
        temp_distance = distance_earth(lat1,lon1,lat2,lon2)
        if temp_distance < closest_distance:
            closest_distance = temp_distance
            closest_county = cnty2
    data_1_2010.append({'county':cnty,'closest_county':closest_county,'closest_distance':closest_distance})


In [15]:
df1_1_2010 = pd.DataFrame(data_1_2010)

In [16]:
#df1_1_2010.sort_values(['closest_county'])
df1_1_2010 = df1_1_2010[['closest_county','county','closest_distance']].sort_values(['closest_county'])

In [17]:
df1_1_2010.head()
#groupby()

,closest_county,county,closest_distance
101,angelina,trinity,32.802964
81,angelina,polk,34.847860
85,aransas,refugio,13.003857
11,aransas,calhoun,26.310291
105,austin,washington,25.240788


In [18]:
df1_1_2010.closest_county.unique().shape

(70,)

### 2015 data

In [19]:
no_clc_list = list(clnc_df[clnc_df['totclin2015'] == 0]['county'])
clc_list = list(clnc_df[clnc_df['totclin2015'] >0 ]['county'])
len(no_clc_list)+len(clc_list)

254

In [20]:
data_1_2015 = []
for cnty in no_clc_list:
    #print(cnty)
    #temp_cnty = cnty
    lat1 = county_detail[county_detail['county'] == cnty]['latitude'].values[0]
    lon1 = county_detail[county_detail['county'] == cnty]['longitude'].values[0]
    closest_distance = 1000
    for cnty2 in clc_list:
        lat2 = county_detail[county_detail['county'] == cnty2]['latitude'].values[0]
        lon2 = county_detail[county_detail['county'] == cnty2]['longitude'].values[0]
        temp_distance = distance_earth(lat1,lon1,lat2,lon2)
        if temp_distance < closest_distance:
            closest_distance = temp_distance
            closest_county = cnty2
    data_1_2015.append({'county':cnty,'closest_county':closest_county,'closest_distance':closest_distance})


In [21]:
df1_1_2015 = pd.DataFrame(data_1_2015)

In [22]:
df1_1_2015 = df1_1_2015[['closest_county','county','closest_distance']].sort_values(['closest_county'])

In [23]:
df1_1_2015.head()

,closest_county,county,closest_distance
94,aransas,refugio,13.003857
10,aransas,calhoun,26.310291
20,austin,colorado,24.971734
69,bailey,lamb,27.420050
22,bell,coryell,31.087835


In [24]:
df1_1_2015.shape

(126, 3)

In [25]:
df1_1_2015.closest_county.unique().shape

(65,)

### Output files

In [26]:
# df1_1_2015.to_csv('data_2015.csv')
# df1_1_2010.to_csv('data_2010.csv')


In [27]:
# county_detail.to_csv('data_county_centroid.csv')

### Calculate new catchment populations

**2010**

In [28]:
clinic.columns[:5]

Index(['county', 'fempop2015_ACS', 'fempop2010_ACS', 'fempop2010_TDC',
       'fempop2015_TDC'],
      dtype='object')

In [29]:
pop = clinic[['county','fempop2015_TDC', 'fempop2010_TDC']]


In [30]:
df1_1_2010.columns

Index(['closest_county', 'county', 'closest_distance'], dtype='object')

In [31]:
df1_1_2010.head()

,closest_county,county,closest_distance
101,angelina,trinity,32.802964
81,angelina,polk,34.847860
85,aransas,refugio,13.003857
11,aransas,calhoun,26.310291
105,austin,washington,25.240788


In [32]:
pop[['county','fempop2010_TDC']].head()

,county,fempop2010_TDC
0,anderson,8184
1,andrews,2922
2,angelina,17078
3,aransas,3309
4,archer,1445


In [33]:
df1_catch_2010 = df1_1_2010.merge(pop[['county','fempop2010_TDC']],how='inner',on='county')

In [34]:
df2_catch_2010 = df1_catch_2010[['closest_county','fempop2010_TDC']].groupby('closest_county').sum().reset_index()

In [35]:
df2_catch_2010.head()

,closest_county,fempop2010_TDC
0,angelina,8872
1,aransas,5048
2,austin,9254
3,baylor,1877
4,bee,2758


In [36]:
# expand to list that of counties with clinic, but that do not have catchment counties
clc_list = list(clnc_df[clnc_df['totclinic2010'] >0 ]['county'])
# 
no_catch_list = list(set(clc_list) - set(df2_catch_2010.closest_county.to_list()))

In [37]:
# create extention data frame, then concat
df2_temp_2010 = pd.DataFrame(list( zip(no_catch_list,[0]*len(no_catch_list) ) ),columns=list(df2_catch_2010.columns))


In [38]:
df2_temp_2010.head()

,closest_county,fempop2010_TDC
0,bell,0
1,caldwell,0
2,cochran,0
3,winkler,0
4,orange,0


In [39]:
df3_2010 = pd.concat([df2_catch_2010,df2_temp_2010],ignore_index=True)#,ignore_index=True)

In [40]:
df3_2010.columns = ['county','fempop_catch_2010']

**2015**

In [53]:
df1_catch_2015 = df1_1_2015.merge(pop[['county','fempop2015_TDC']],how='inner',on='county')

In [54]:
df2_catch_2015 = df1_catch_2015[['closest_county','fempop2015_TDC']].groupby('closest_county').sum().reset_index()

In [55]:
df2_catch_2015.head()

,closest_county,fempop2015_TDC
0,aransas,5001
1,austin,3327
2,bailey,2268
3,bell,18068
4,brooks,907


In [56]:
# expand to list that of counties with clinic, but that do not have catchment counties
clc_list = list(clnc_df[clnc_df['totclin2015'] >0 ]['county'])
no_catch_list = list(set(clc_list) - set(df2_catch_2015.closest_county.to_list()))

In [57]:
# create extention data frame, then concat
df2_temp_2015 = pd.DataFrame(list( zip(no_catch_list,[0]*len(no_catch_list) ) ),columns=list(df2_catch_2015.columns))


In [58]:
df3_2015 = pd.concat([df2_catch_2015,df2_temp_2015],ignore_index=True)#,ignore_index=True)

In [59]:
df3_2015.columns = ['county','fempop_catch_2015']

In [60]:
df3_2015.head()

,county,fempop_catch_2015
0,aransas,5001
1,austin,3327
2,bailey,2268
3,bell,18068
4,brooks,907


**Update Master Clinic Table**<br>
New fields:<br>
* 2010 Catchment Fempop - fpop_cat_2010
* 2010 Total Fempop - fpop_tot_2010
* 2015 Catchment Fempop  - fpop_cat_2015
* 2015 Total Fempop - fpop_tot_2015



In [61]:
clinic2 = clinic.merge(df3_2010,how='left',on='county')


In [62]:
clinic3 = clinic2.merge(df3_2015,how='left',on='county')


In [63]:
temp_2010 = clinic3['fempop_catch_2010'] + clinic3['fempop2010_TDC']
temp_2015 = clinic3['fempop_catch_2015'] + clinic3['fempop2015_TDC']

In [64]:
clinic3['fempop_tot_2010'] = temp_2010
clinic3['fempop_tot_2015'] = temp_2015

In [65]:
clinic3.head()

,county,fempop2015_ACS,fempop2010_ACS,fempop2010_TDC,fempop2015_TDC,whitefempop2015_TDC,blackfempop2015_TDC,otherfempop2015_TDC,hispfempop2015_TDC,job2015all,...,pccm2010,mco2010,Medtot2010,HSR (8),border,metro,fempop_catch_2010,fempop_catch_2015,fempop_tot_2010,fempop_tot_2015
0,anderson,7811.208,8768.696,8184,8017,4954,1171,278,1614,41.4,...,322,0,359,3,0,1,NaN,NaN,NaN,NaN
1,andrews,3237.892,2774.236,2922,3449,1380,41,63,1965,64.8,...,104,0,111,7,0,1,0.0,0.0,2922.0,3449.0
2,angelina,17227.008,16998.540,17078,17139,9264,2904,495,4476,54.2,...,641,0,680,3,0,1,8872.0,NaN,25950.0,NaN
3,aransas,3518.907,3753.496,3309,3373,1909,29,151,1284,46.7,...,7,143,158,8,0,1,5048.0,5001.0,8357.0,8374.0
4,archer,1453.650,1504.368,1445,1546,1351,7,26,162,59.7,...,14,0,15,2,0,2,NaN,NaN,NaN,NaN


In [66]:
#clinic3.to_excel('clinic_data.xlsx',index=False)

### to DO

* add a closest clinic column for each time period

**Create two new columns**<br>
* distance_toclincic_2010
* distance_toclincic_2015

In [45]:
temp = df1_1_2010[['county', 'closest_distance']]
temp.columns =['county', 'distance_toclincic_2010']

In [46]:
temp.head()

,county,distance_toclincic_2010
101,trinity,32.802964
81,polk,34.847860
85,refugio,13.003857
11,calhoun,26.310291
105,washington,25.240788


In [67]:
clinic3 = clinic3.merge(temp,how='left',on='county')

In [70]:
temp = df1_1_2015[['county', 'closest_distance']]
temp.columns =['county', 'distance_toclincic_2015']


In [71]:
clinic3 = clinic3.merge(temp,how='left',on='county')

In [72]:
clinic3.head()

,county,fempop2015_ACS,fempop2010_ACS,fempop2010_TDC,fempop2015_TDC,whitefempop2015_TDC,blackfempop2015_TDC,otherfempop2015_TDC,hispfempop2015_TDC,job2015all,...,Medtot2010,HSR (8),border,metro,fempop_catch_2010,fempop_catch_2015,fempop_tot_2010,fempop_tot_2015,distance_toclincic_2010,distance_toclincic_2015
0,anderson,7811.208,8768.696,8184,8017,4954,1171,278,1614,41.4,...,359,3,0,1,NaN,NaN,NaN,NaN,28.589673,28.589673
1,andrews,3237.892,2774.236,2922,3449,1380,41,63,1965,64.8,...,111,7,0,1,0.0,0.0,2922.0,3449.0,NaN,NaN
2,angelina,17227.008,16998.540,17078,17139,9264,2904,495,4476,54.2,...,680,3,0,1,8872.0,NaN,25950.0,NaN,NaN,20.794900
3,aransas,3518.907,3753.496,3309,3373,1909,29,151,1284,46.7,...,158,8,0,1,5048.0,5001.0,8357.0,8374.0,NaN,NaN
4,archer,1453.650,1504.368,1445,1546,1351,7,26,162,59.7,...,15,2,0,2,NaN,NaN,NaN,NaN,26.615393,26.615393


In [73]:
clinic3.to_excel('clinic_data_2_26.xlsx',index=False)